In [11]:
import pandas as pd

In [13]:
uhauldata = pd.read_csv("../data/uhauloutput-35cities.csv")

In [18]:
incomedata = pd.read_csv("../data/income.csv")

In [21]:
incomedata = incomedata.drop("Unnamed: 0", axis=1)

In [22]:
uhauldata_filtered = uhauldata[
    (uhauldata['Pickup Location'].isin(incomedata["City"].unique())) &
    (uhauldata['Dropoff Location'].isin(incomedata["City"].unique()))
]

In [23]:
uhauldata_filtered

,Pickup Location,Dropoff Location,Rate,Truck Type,Moving Type,Date
0,"Atlanta, GA","Boston, MA","$2,111.00",10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
1,"Atlanta, GA","Boston, MA","$2,223.00",15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
2,"Atlanta, GA","Boston, MA","$2,443.00",20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024
3,"Atlanta, GA","Boston, MA","$2,779.00",26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
8,"Atlanta, GA","Charlotte, NC",$429.00,10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
...,...,...,...,...,...,...
4736,"Wichita, KS","Tulsa, OK",$375.00,26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
4737,"Wichita, KS","Washington, DC","$1,749.00",10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
4738,"Wichita, KS","Washington, DC","$1,841.00",15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
4739,"Wichita, KS","Washington, DC","$2,209.00",20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024


In [33]:
uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].str.replace('$', '', regex=False)


/var/folders/_k/twyqm8dd3gvdw3vfzr0hk_5r0000gn/T/ipykernel_75888/849234699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].str.replace('$', '', regex=False)


In [41]:
uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].replace(',', '', regex=True).astype(float)

/var/folders/_k/twyqm8dd3gvdw3vfzr0hk_5r0000gn/T/ipykernel_75888/3529183148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].replace(',', '', regex=True).astype(float)


In [44]:
rate_sum_by_ending_city = uhauldata_filtered.groupby('Dropoff Location')['Rate'].mean().reset_index()
rate_sum_by_ending_city

,Dropoff Location,Rate
0,"Albuquerque, NM",1696.903226
1,"Atlanta, GA",1639.752066
2,"Austin, TX",1840.911290
3,"Baltimore, MD",1789.062016
4,"Boston, MA",2094.769231
5,"Charlotte, NC",1938.523438
6,"Chicago, IL",1564.512195
7,"Cincinnati, OH",1458.453125
8,"Cleveland, OH",1571.585938
9,"Columbus, OH",1525.781250


In [46]:
truckindex = pd.read_csv("../data/truckindex.csv")


In [47]:
merged_df = rate_sum_by_ending_city.assign(key=1).merge(truckindex.assign(key=1), on='key').drop('key', axis=1)

In [48]:
merged_df['TruckingCost'] = merged_df['Rate'] * merged_df['PriceIndex']

In [52]:

# Strip whitespace from column names (if any)
merged_df.columns = merged_df.columns.str.strip()

# Step 4: Create Final DataFrame with trucking costs
final_df = merged_df[['Dropoff Location', 'Year', 'TruckingCost']]

In [53]:
final_df

,Dropoff Location,Year,TruckingCost
0,"Albuquerque, NM",2005,2110.947613
1,"Albuquerque, NM",2006,2183.914452
2,"Albuquerque, NM",2007,2136.401161
3,"Albuquerque, NM",2008,1914.106839
4,"Albuquerque, NM",2009,1568.787032
...,...,...,...
622,"Wichita, KS",2019,1875.071344
623,"Wichita, KS",2020,1730.099813
624,"Wichita, KS",2021,1899.764297
625,"Wichita, KS",2022,1916.491781


In [54]:
final_df.to_csv("../data/truckcost.csv")